In [1]:
import arcpy
from timeit import default_timer as timer

print("Start Operations")
start = timer()

#parameters to set
gdb_working = r"D:\ArcGIS_Local\LiDAR\Contours.gdb"
fd_contour_inputs = fr"{gdb_working}\ContourInputs"
fc_watershed = "cHUC12_Enriched"
fc_grid = "cUSNG_1000m"
#assumes featureclasses named like "Contours_YEAR"
contour_years = ["2001", "2007", "2011", "2015", "2019", "2020"]

arcpy.env.parallelProcessingFactor = "100%"
arcpy.env.overwriteOutput = True

for contour_year in contour_years:
    print(f"\nBeginning Routine for Contour Year {contour_year}")
    start = timer()

    # variables
    fc_contours = f"Contours_{contour_year}"

    # load featureclasses into memory
    print("\tLoading Watersheds and USNG into Memory")
    arcpy.env.workspace = fd_contour_inputs
    mem_watersheds = arcpy.conversion.FeatureClassToFeatureClass(
        fc_watershed, "memory", "mem_Watershed"
    )
    mem_grid = arcpy.conversion.FeatureClassToFeatureClass(fc_grid, "memory", "mem_Grid")
    print("\tLoading Contours into Memory")
    mem_contours = arcpy.conversion.FeatureClassToFeatureClass(
        fc_contours, "memory", f"mem_{fc_contours}"
    )

    # set env to memory
    arcpy.env.workspace = r"in_memory"
    arcpy.env.parallelProcessingFactor = "100%"
    arcpy.env.overwriteOutput = True

    # run intersect
    print("\tBeginning Intersect Routine")
    mem_multi_intersect01 = arcpy.analysis.Intersect(
        [mem_contours, mem_watersheds], "mem_multi_intersect01", output_type="POINT"
    )
    print("\t\tIntersect 1 Complete - Contours Intersected Against Watersheds")
    mem_multi_intersect02 = arcpy.analysis.Intersect(
        [mem_contours, mem_grid], "mem_multi_intersect02", output_type="POINT"
    )
    print("\t\tIntersect 2 Complete - Contours Intersected Against USNG Grid")

    print("\tAppending Intersects to single featureclass")
    arcpy.management.Append(
        inputs=mem_multi_intersect02, target=mem_multi_intersect01, schema_type="NO_TEST"
    )
    print("\tConverting multi-point to single-point featureclass")
    mem_point_intersects = arcpy.management.MultipartToSinglepart(
        mem_multi_intersect01, f"mem_point_intersects"
    )
    print("\tSplitting Contours at Intersected Points")
    mem_contours_split = arcpy.management.SplitLineAtPoint(
        mem_contours, mem_point_intersects, f"mem_split_contours", "10 Feet"
    )
    print("\tSpatially Joining Contours for Enrichment")
    print("\t\tJoining Contours Against Watersheds")
    mem_contours_sj_water = arcpy.analysis.SpatialJoin(target_features=mem_contours_split, join_features=mem_watersheds, out_feature_class="mem_contours_sj_water", join_operation="JOIN_ONE_TO_MANY", join_type="KEEP_ALL", match_option="HAVE_THEIR_CENTER_IN")
    print("\t\tJoining Contours Against USNG")
    mem_contours_sj_grid = arcpy.analysis.SpatialJoin(target_features=mem_contours_sj_water, join_features=mem_grid, out_feature_class="mem_contours_sj_grid", join_operation="JOIN_ONE_TO_MANY", join_type="KEEP_ALL", match_option="HAVE_THEIR_CENTER_IN")
    print("\tSorting Data")
    mem_sorted = arcpy.management.Sort(mem_contours_sj_grid, f"mem_Contours_Final", "USNG ASCENDING;HUC12 ASCENDING;Contour ASCENDING;Shape_Length ASCENDING", "LL")
    print("\tDeleting Fields")
    arcpy.management.DeleteField(mem_sorted, "Join_Count;TARGET_FID;JOIN_FID;Join_Count_1;TARGET_FID_1;JOIN_FID_1;ORIG_FID;ORIG_SEQ;created_user;created_date;last_edited_user;last_edited_date;Shape_Length_1", "DELETE_FIELDS")
    print("\tExporting to FGDB")
    arcpy.conversion.FeatureClassToFeatureClass(mem_sorted, fr"{gdb_working}\ContourYears", f"Contours_{contour_year}_Final")
    end = timer()
    print(f"\tRoutine for Contour Year {contour_year} in {(((end - start)/60)/60):.2f} hours")
    arcpy.Delete_management("memory")


print("All Operations Complete")

Start Operations

Beginning Routine for Contour Year 2001
	Loading Watersheds and USNG into Memory
	Loading Contours into Memory
	Beginning Intersect Routine
		Intersect 1 Complete - Contours Intersected Against Watersheds
		Intersect 2 Complete - Contours Intersected Against USNG Grid
	Appending Intersects to single featureclass
	Converting multi-point to single-point featureclass
	Splitting Contours at Intersected Points
	Spatially Joining Contours for Enrichment
		Joining Contours Against Watersheds
		Joining Contours Against USNG
	Sorting Data
	Deleting Fields
	Exporting to FGDB
	Routine for Contour Year 2001 in 0.08 hours

Beginning Routine for Contour Year 2007
	Loading Watersheds and USNG into Memory
	Loading Contours into Memory
	Beginning Intersect Routine
		Intersect 1 Complete - Contours Intersected Against Watersheds
		Intersect 2 Complete - Contours Intersected Against USNG Grid
	Appending Intersects to single featureclass
	Converting multi-point to single-point featurecla